# <font style="color:blue">2. Generating Anchor Boxes</font>

Our detector has 9 anchors for every feature map by default.

<img src='https://www.learnopencv.com/wp-content/uploads/2020/03/c3-w8-anchors.png' align='middle'>

**What is feature map here?**

Let's say `a` is an input image of dimension `256x256`, and it has two feature maps `b` (`8 x 8 feature map (grid)`) and `c` (`4 x 4 feature map (grid)`). One element of the feature map represents segments of pixels in the original image `a`.



**Why 9?**

To answer this question, let's take a look into DataEncoder class.
We have 3 aspect ratios of sizes $1/2$, $1$ and $2$. For each size, there are `3` scales: $1, 2^{1/3}$ and $2^{2/3}$.
These anchors of the appropriate sizes are generated for each of five feature maps we have.

In [4]:
from IPython.display import Code
import inspect

from trainer.encoder import (
    DataEncoder,
    decode_boxes,
    encode_boxes,
    generate_anchors,
    generate_anchor_grid
)

In [10]:
import math

In [11]:
class DataEncoder:
    def __init__(self, input_size):
        self.input_size = input_size
        self.anchor_areas = [8 * 8, 16 * 16., 32 * 32., 64 * 64., 128 * 128]  # p3 -> p7
        self.aspect_ratios = [0.5, 1, 2]
        self.scales = [1, pow(2, 1 / 3.), pow(2, 2 / 3.)]
        num_fms = len(self.anchor_areas)
        fm_sizes = [math.ceil(self.input_size[0] / pow(2., i + 3)) for i in range(num_fms)]
        print(fm_sizes)
        self.anchor_boxes = []
        for i, fm_size in enumerate(fm_sizes):
            anchors = generate_anchors(self.anchor_areas[i], self.aspect_ratios, self.scales)
            anchor_grid = generate_anchor_grid(input_size, fm_size, anchors)
            self.anchor_boxes.append(anchor_grid)
        self.anchor_boxes = torch.cat(self.anchor_boxes, 0)
        self.classes = ["__background__", "person"]

    def encode(self, boxes, classes):
        iou = compute_iou(boxes, self.anchor_boxes)
        iou, ids = iou.max(1)
        loc_targets = encode_boxes(boxes[ids], self.anchor_boxes)
        cls_targets = classes[ids]
        cls_targets[iou < 0.5] = -1
        cls_targets[iou < 0.4] = 0

        return loc_targets, cls_targets

    def decode(self, loc_pred, cls_pred, cls_threshold=0.7, nms_threshold=0.3):
        all_boxes = [[] for _ in range(len(loc_pred))]  # batch_size

        for sample_id, (boxes, scores) in enumerate(zip(loc_pred, cls_pred)):
            boxes = decode_boxes(boxes, self.anchor_boxes)

            conf = scores.softmax(dim=1)
            sample_boxes = [[] for _ in range(len(self.classes))]
            for class_idx, class_name in enumerate(self.classes):
                if class_name == '__background__':
                    continue
                class_conf = conf[:, class_idx]
                ids = (class_conf > cls_threshold).nonzero().squeeze()
                ids = [ids.tolist()]
                keep = compute_nms(boxes[ids], class_conf[ids], threshold=nms_threshold)

                conf_out, top_ids = torch.sort(class_conf[ids][keep], dim=0, descending=True)
                boxes_out = boxes[ids][keep][top_ids]

                boxes_out = boxes_out.cpu().numpy()
                conf_out = conf_out.cpu().numpy()

                c_dets = np.hstack((boxes_out, conf_out[:, np.newaxis])).astype(np.float32, copy=False)
                c_dets = c_dets[c_dets[:, 4].argsort()]
                sample_boxes[class_idx] = c_dets

            all_boxes[sample_id] = sample_boxes

        return all_boxes

    def get_num_anchors(self):
        return len(self.aspect_ratios) * len(self.scales)

**Why have we chosen the following anchor area?**
```
anchor_areas = [8 * 8, 16 * 16., 32 * 32., 64 * 64., 128 * 128]  # p3 -> p7
```
The first anchor area is responsible for generating anchors for the first output layer of `FPN` and so on. 

```
256/8 = 32

256/16 = 16
   .
   .
256/128 = 2
```

**So, how do we generate them?**

We first generate our 9 anchors, knowing which areas it should cover, using predefined ratios and scales.

In [12]:
def generate_anchors(anchor_area, aspect_ratios, scales):
    anchors = []
    for scale in scales:
        for ratio in aspect_ratios:
            h = math.sqrt(anchor_area/ratio)
            w = math.sqrt(anchor_area*ratio)
            x1 = (math.sqrt(anchor_area) - scale * w) * 0.5
            y1 = (math.sqrt(anchor_area) - scale * h) * 0.5
            x2 = (math.sqrt(anchor_area) + scale * w) * 0.5
            y2 = (math.sqrt(anchor_area) + scale * h) * 0.5
            anchors.append([x1, y1, x2, y2])
    return torch.Tensor(anchors)

For each feature map we create a grid, that will allow us to densely put all of the possible boxes.

In [13]:
def generate_anchor_grid(input_size, fm_size, anchors):
    grid_size = input_size[0] / fm_size
    x, y = torch.meshgrid(torch.arange(0, fm_size) * grid_size, torch.arange(0, fm_size) * grid_size)
    anchors = anchors.view(-1, 1, 1, 4)
    xyxy = torch.stack([x, y, x, y], 2).float()
    boxes = (xyxy + anchors).permute(2, 1, 0, 3).contiguous().view(-1, 4)
    boxes[:, 0::2] = boxes[:, 0::2].clamp(0, input_size[0])
    boxes[:, 1::2] = boxes[:, 1::2].clamp(0, input_size[1])
    return boxes

**Let's check the size of the anchor boxes. for input image size `3x256x256` and `3x300x300`.**

In [14]:
height_width = (256, 256)

data_encoder = DataEncoder(height_width)

print('anchor_boxes size: {}'.format(data_encoder.anchor_boxes.size()))

[32, 16, 8, 4, 2]


NameError: name 'torch' is not defined

In [6]:
height_width = (300, 300)

data_encoder = DataEncoder(height_width)

print('anchor_boxes size: {}'.format(data_encoder.anchor_boxes.size()))

anchor_boxes size: torch.Size([17451, 4])


**Let's compare anchor boxes size to detector network output size:**

<div>
    <table>
        <tr><td><h3>Image input size</h3></td> <td><h3>Anchor boxes size</h3></td> <td><h3>Detector Network output size</h3></td> </tr>
        <tr><td><h3>(256, 256)</h3></td> <td><h3>[12276, 4]</h3></td> <td><h3>[batch_size, 12276, 4]</h3></td> </tr>
        <tr><td><h3>(300, 300)</h3></td> <td><h3>[17451, 4]</h3></td> <td><h3>[batch_size, 17451, 4]</h3></td> </tr>
    </table>
</div>

Basically, we want to encode our location target, such that the size location target becomes equal to the size of anchor boxes.
